<a href="https://colab.research.google.com/github/JinnyYam/study/blob/main/%08HW2_2014002_%EC%9D%B4%EC%9C%A0%EC%A7%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 유전자 알고리즘

###수정사항
1. 돌연변이 확률 증가
2. 유전자 개수 변경 -> 6
3. 적합도 함수(cal_fitness) 변경 

   -> 음수의 값이 나오면, 그 수의 절대값의 역수를 취해서 적합값으로 만들어서 뽑힐 확률을 낮춰줌
4. 메인 프로그램의 반복문 조건 변경함

In [93]:
"""
### 수정사항
### 1. 돌연변이 확률 증가 (0.1 -> 0.2)
### 2. 유전자 개수 변경 (5 -> 6)
### 3. 적합도 함수(cal_fitness) 변경 -> 음수의 값이 나오면, 그 수의 절대값의 역수를 취해서 적합값으로 만들어서 뽑힐 확률을 낮춰줌
### 4. 메인 프로그램의 반복문 조건 변경함
"""


import random

POPULATION_SIZE = 4		    # 개체 집단의 크기
MUTATION_RATE = 0.2	    # 돌연 변이 확률
SIZE = 6				    # 하나의 염색체에서의 유전자 개수		

# 염색체를 클래스로 정의한다. 
class Chromosome:
    def __init__(self, g=[]):
        self.genes = g.copy()		# 유전자를 리스트로 구현된다. 
        self.fitness = 0		    # 적합도
        if self.genes.__len__()==0:	# 염색체가 초기 상태이면 초기화한다. 
            i = 0
            while i < SIZE:
                if random.random() >= 0.5:      # 0~1 사이의 실수 난수
                    self.genes.append(1) 
                else: 
                    self.genes.append(0)
                i += 1
    
    def cal_fitness(self):		    # 적합도 계산 
        self.fitness = 0
        value = 0
        for i in range(SIZE):
            value += self.genes[i]*pow(2, SIZE-1-i)

        result = -pow(value,3)+60*pow(value,2)-800*value  # 문제에 주어진 수식에 넣은 결과값
        if(result<0): # 음수가 들어오면 절대값의 역수를 취해서 넣어줌
            self.fitness = 1/abs(result)
        else: # 0이상이 들어오면 그대로 넣어줌
            self.fitness = result
        return self.fitness
    

    def __str__(self):
        return self.genes.__str__()

# 개체 속의 염색체와 염색체의 적합도를 출력한다. 
def print_p(pop):
    i = 0
    for x in pop:
        print("염색체 #", i, "=", x, "적합도=", x.cal_fitness())
        i += 1
    print("")

# 선택 연산
def select(pop):
    max_value  = sum([c.cal_fitness() for c in population])
    pick = random.uniform(0, max_value)  # 0과 max_value 사이의 실수 발생
    current = 0
    print("Fitness 총합 = ", max_value, ",Pick = ", pick)
    
    # 룰렛휠에서 어떤 조각에 속하는지를 알아내는 루프
    for c in pop:
        current += c.cal_fitness()
        if current > pick:
            return c


# 교차 연산
def crossover(pop):
    father = select(pop)
    mother = select(pop)
    index = random.randint(1, SIZE - 1)     # 1~5의 인덱스 선택, 슬라이싱에 사용
    child1 = father.genes[:index] + mother.genes[index:] 
    child2 = mother.genes[:index] + father.genes[index:] 
    return (child1, child2)
    
# 돌연변이 연산
def mutate(c):
    for i in range(SIZE):
        if random.random() < MUTATION_RATE:
            if random.random() < 0.5:
                c.genes[i] = 1
            else:
                c.genes[i] = 0

In [104]:
# 메인 프로그램
population = []
i=0

# 초기 염색체를 생성하여 객체 집단에 추가한다. 
while i < POPULATION_SIZE:
    population.append(Chromosome())
    i += 1
    
count=0
print("+++++++++++++++++++++++++++++++++++++++")
print("세대 번호:", count)
population.sort(key=lambda x: x.cal_fitness(), reverse=True)
print_p(population)

count=1
while population[0].cal_fitness() < 3096: # x가 31일때 최대 값 3069가 나온다.
    new_pop = []

    # 선택과 교차 연산
    for _ in range(POPULATION_SIZE//2):     #  몫 연산자
        c1, c2 = crossover(population)
        new_pop.append(Chromosome(c1))
        new_pop.append(Chromosome(c2))

    # 자식 세대가 부모 세대를 대체함, 깊은 복사 수행 
    population = new_pop.copy();
    print()
    print("** 교차연산 후")
    print_p(population)
    
    # 돌연변이 연산
    for c in population: 
        mutate(c)
    print("** 돌연변이 후")
    print_p(population)
    
    # 출력을 위한 정렬
    print("+++++++++++++++++++++++++++++++++++++++")
    population.sort(key=lambda x: x.cal_fitness(), reverse=True)
    print("세대 번호:", count)
    print_p(population)
    count += 1
    if count > 20: 
        break

+++++++++++++++++++++++++++++++++++++++
세대 번호: 0
염색체 # 0 = [1, 0, 0, 1, 0, 1] 적합도= 1887
염색체 # 1 = [1, 0, 0, 1, 1, 0] 적합도= 1368
염색체 # 2 = [1, 0, 1, 0, 0, 1] 적합도= 0.0011614401858304297
염색체 # 3 = [0, 1, 0, 0, 0, 0] 적합도= 0.0006510416666666666

Fitness 총합 =  3255.0018124818525 ,Pick =  449.9307616364501
Fitness 총합 =  3255.0018124818525 ,Pick =  49.01175732322855
Fitness 총합 =  3255.0018124818525 ,Pick =  3077.6555155973156
Fitness 총합 =  3255.0018124818525 ,Pick =  3006.2222399329826

** 교차연산 후
염색체 # 0 = [1, 0, 0, 1, 0, 1] 적합도= 1887
염색체 # 1 = [1, 0, 0, 1, 0, 1] 적합도= 1887
염색체 # 2 = [1, 0, 0, 1, 1, 0] 적합도= 1368
염색체 # 3 = [1, 0, 0, 1, 1, 0] 적합도= 1368

** 돌연변이 후
염색체 # 0 = [1, 0, 0, 1, 0, 1] 적합도= 1887
염색체 # 1 = [1, 0, 0, 1, 1, 1] 적합도= 741
염색체 # 2 = [1, 0, 0, 1, 1, 1] 적합도= 741
염색체 # 3 = [1, 0, 0, 1, 1, 0] 적합도= 1368

+++++++++++++++++++++++++++++++++++++++
세대 번호: 1
염색체 # 0 = [1, 0, 0, 1, 0, 1] 적합도= 1887
염색체 # 1 = [1, 0, 0, 1, 1, 0] 적합도= 1368
염색체 # 2 = [1, 0, 0, 1, 1, 1] 적합도= 741
염색체 # 3 = [1, 0, 0, 